### 10 Python Pandas tricks to make data analysis more enjoyable

출처  
https://towardsdatascience.com/10-python-pandas-tricks-to-make-data-analysis-more-enjoyable-cb8f55af8c30  
https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

In [4]:
import pandas as pd
import numpy as np

#### 1. Styling

특정조건(음수값) 다른 색깔 주기

In [1]:
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color
df = pd.DataFrame(dict(col_1=[1.53,-2.5,3.53], 
                       col_2=[-4.1,5.9,0])
                 )
df.style.applymap(color_negative_red)

,col_1,col_2
0,1.53,-4.1
1,-2.5,5.9
2,3.53,0


인덱스 없애기

In [2]:
df.head().style.hide_index()

col_1,col_2
1.53,-4.1
-2.5,5.9
3.53,0


표에 마우스 오버(hover) 효과 주기

In [5]:
df = pd.DataFrame(np.random.randn(5, 3))

df.style.set_table_styles(
[{'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}]
)

,0,1,2
0,-1.93302,0.543243,-0.808563
1,-1.42373,-0.59664,-1.96142
2,2.49587,1.16045,-0.0551432
3,-0.0582352,-0.41949,-0.893241
4,0.251587,1.07996,-0.121987


스타일 시트(CSS) 효과 주기

In [7]:
df = pd.DataFrame(
dict(departure=['SFO', 'SFO', 'LAX', 'LAX', 'JFK', 'SFO'],
     arrival=['ORD', 'DFW', 'DFW', 'ATL', 'ATL', 'ORD'],
     airlines=['Delta','JetBlue','Delta','AA','SouthWest',  
               'Delta']),
columns=['airlines', 'departure','arrival'])

df.style.set_table_styles(
[{'selector': 'tr:nth-of-type(odd)',
  'props': [('background', '#eee')]}, 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 {'selector': 'th',
  'props': [('background', '#606060'), 
            ('color', 'white'),
            ('font-family', 'verdana')]},
 {'selector': 'td',
  'props': [('font-family', 'verdana')]},
]
).hide_index()

airlines,departure,arrival
Delta,SFO,ORD
JetBlue,SFO,DFW
Delta,LAX,DFW
AA,LAX,ATL
SouthWest,JFK,ATL
Delta,SFO,ORD


#### 2. Pandas Options

In [ ]:
pd.options.display.max_columns = 50  # None -> No Restrictions
pd.options.display.max_rows = 200    # None -> Be careful with this 
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

#### 3. Group by with multiple aggregations

In [25]:
df = pd.DataFrame(dict(A=['coke', 'sprite', 'coke', 'sprite',
                          'sprite', 'coke', 'coke'],
                       B=['alpha','gamma', 'alpha', 'beta',
                          'gamma', 'beta', 'beta'],
                       col_1=[1,2,3,4,5,6,7],
                       col_2=[1,6,2,4,7,9,3]))
tbl = df.groupby(['A','B']).agg({'col_1': ['max', np.mean],
                                 'col_2': ['sum','min','count']})
tbl        # 'count' will always be the count for number of rows in each group.

col_1      col_2          
               max mean   sum min count
A      B                               
coke   alpha     3  2.0     3   1     2
       beta      7  6.5    12   3     2
sprite beta      4  4.0     4   4     1
       gamma     5  3.5    13   6     2

#### 4. Column slicing

In [26]:
df.iloc[:,2:5].head()             # select the 2nd to the 4th column

,col_1,col_2
0,1,1
1,2,6
2,3,2
3,4,4
4,5,7


In [27]:
df.loc[:,'column_x':].head()      # select all columns starting from 'column_x'

""
0
1
2
3
4


#### 5. Add row ID / random row ID to each group

In [28]:
# df: target dataframe 
np.random.seed(0)    # set random seed
df['random_ID_all'] = np.random.permutation(df.shape[0])
df['ID_all'] = [i for i in range(1, df.shape[0]+1)]

In [29]:
df['ID'] = df.groupby(['A', 'B'])['ID_all'].rank(method='first', ascending=True).astype(int)
df['random_ID'] = df.groupby(['A', 'B'])['random_ID_all'].rank(method='first', ascending=True).astype(int)

In [30]:
df

,A,B,col_1,col_2,random_ID_all,ID_all,ID,random_ID
0,coke,alpha,1,1,6,1,1,2
1,sprite,gamma,2,6,2,2,1,2
2,coke,alpha,3,2,1,3,2,1
3,sprite,beta,4,4,3,4,1,1
4,sprite,gamma,5,7,0,5,2,1
5,coke,beta,6,9,5,6,1,2
6,coke,beta,7,3,4,7,2,1


#### 6. List all unique values in group

In [33]:
df = pd.DataFrame(dict(A=['A','A','A','A','A','B','B','B','B'],
                       B=[1,1,1,2,2,1,1,1,2],
                       C=['CA','NY','CA','FL','FL',     
                          'WA','FL','NY','WA']))
df

,A,B,C
0,A,1,CA
1,A,1,NY
2,A,1,CA
3,A,2,FL
4,A,2,FL
5,B,1,WA
6,B,1,FL
7,B,1,NY
8,B,2,WA


In [32]:
tbl = df[['A', 'B', 'C']].drop_duplicates()\
                         .groupby(['A','B'])['C']\
                         .apply(list)\
                         .reset_index()

# list to string (separated by commas) 
tbl['C'] = tbl.apply(lambda x: (','.join([str(s) for s in x['C']])), axis = 1)
tbl

,A,B,C
0,A,1,"CA,NY"
1,A,2,FL
2,B,1,"WA,FL,NY"
3,B,2,WA


#### 7. Add row total and column total to a numerical dataframe

In [34]:
df = pd.DataFrame(dict(A=[2,6,3],
                       B=[2,2,6], 
                       C=[3,2,3]))
df['col_total']     = df.apply(lambda x: x.sum(), axis=1)
df.loc['row_total'] = df.apply(lambda x: x.sum())

In [35]:
df

,A,B,C,col_total
0,2,2,3,7
1,6,2,2,10
2,3,6,3,12
row_total,11,10,8,29


#### 8. Checking memory usage

In [38]:
df.memory_usage(deep=True)

Index        170
A             32
B             32
C             32
col_total     32
dtype: int64

#### 9. Cumulative sum

In [40]:
df['cumulative_sum'] = df['col_total'].cumsum()
df

,A,B,C,col_total,cumulative_sum
0,2,2,3,7,7
1,6,2,2,10,17
2,3,6,3,12,29
row_total,11,10,8,29,58


#### 10. Crosstab

In [44]:
df = pd.DataFrame(dict(departure=['SFO','SFO','LAX','LAX', 'JFK','SFO'],
                       arrival=['ORD','DFW','DFW','ATL','ATL','ORD'],
                       airlines=['Delta','JetBlue','Delta','AA','SouthWest','Delta']))
df

,departure,arrival,airlines
0,SFO,ORD,Delta
1,SFO,DFW,JetBlue
2,LAX,DFW,Delta
3,LAX,ATL,AA
4,JFK,ATL,SouthWest
5,SFO,ORD,Delta


In [49]:
pd.crosstab(index=[df['departure'], df['airlines']],
             columns=[df['arrival']],
             rownames=['departure', 'airlines'],
             colnames=['arrival'],
             margins=True         # add subtotal
            )

arrival              ATL  DFW  ORD  All
departure airlines                     
JFK       SouthWest    1    0    0    1
LAX       AA           1    0    0    1
          Delta        0    1    0    1
SFO       Delta        0    0    2    2
          JetBlue      0    1    0    1
All                    2    2    2    6